In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from statsmodels.tsa.stattools import adfuller
from datetime import datetime
import warnings
import glob
warnings.filterwarnings("ignore")

In [21]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime

# Handle different dates
def handle_dates(df):
    if 'date' in df.columns:
        return df
    if 'Date' in df.columns:
            df = df.rename(columns={"Date":"date"})
    # Use cons_start_date insted of fe_fp_end when available
    elif 'fe_fp_end' in df.columns:
        if 'cons_start_date' in df.columns:
            if 'cons_end_date' in df.columns:
                df = df.drop(['fe_fp_end', 'cons_end_date'],axis = 1)
            else:
                df = df.drop(['fe_fp_end'],axis = 1)
            df = df.rename(columns={"cons_start_date":"date"})
        else: 
            if 'guidance_date' in df.columns:
                df = df.drop(['guidance_date'],axis = 1)
            df = df.rename(columns={"fe_fp_end":"date"}) 
    return df
# Quickly turn loaded csv into the desired format
def reindex(df):
    df = df.reset_index().drop(['index','Unnamed: 0','level_0'],axis=1, errors='ignore').set_index('date')
    df.index = pd.to_datetime(df.index)
    df = df.reset_index().set_index('fsym_id').sort_values(by='date').sort_index(kind='mergesort').reset_index().set_index(['fsym_id','date'])
    return df

# List of years we want data from
desired_years = ['2017','2018','2019','2020','2021']
# Load only stocks present in the file below, today's file for intra-day
high_priority = pd.read_excel('Prices/dataset template8k2021.xlsx').melt(id_vars=["Dates"], var_name="fsym_id", value_name="Price")
full_df = pd.DataFrame()
for year in desired_years:
    metric = glob.glob(f'Prices/{year}.csv')[0]
    print(metric)
    df = pd.read_csv(metric, header=4)
    #print(df)
    df = pd.melt(df, id_vars='ticker', var_name='date', value_name='price').rename(columns={"ticker":"fsym_id"})
    df = reindex(df).reset_index()
    # Replace Ticker with fsym_id
    if 'fsym_id' not in df.columns:
        if 'ticker' in df.columns:
            matching = pd.read_csv("fsym_to_bbg_matches.csv")
            df = df.merge(matching, on='ticker', how='left')
            df = df.dropna(subset =['fsym_id']).drop('ticker',axis = 1)
    # Only work with certain stocks
    df = df[df['fsym_id'].isin(high_priority['fsym_id'])]
    # Load next file if current doesn't contain data on stocks we want
    if df.shape[0] == 0: 
        print('No data for desired stocks found, moving to next file.')
        continue
    # Handle different types of date columns: "Date"/"fe_fp_end"/"year-month"
    df = handle_dates(df)
    if 'date' not in df.columns: 
        print('Unrecognizable date format, moving to next file.')
        continue
    df = df.set_index('date')
    df.index = pd.to_datetime(df.index)
    # Change all column names to lowercase
    df.columns = map(str.lower, df.columns)
    # Drop "#NAME?" entries from dataframe
    df = df.replace(['-','#NAME?'],np.nan)
    # Merge current data frame with complete one
    if 'fsym_id' in full_df.reset_index():
        full_df = full_df.merge(df, on=['fsym_id','date'], how='outer')
    else:
        full_df = df
    # Merge repeated columns of the foormat "column_x" and "column_y" into "column"
    for column in df.columns:
        if f'{column}_x' in full_df.columns:
            full_df[f"{column}"] = full_df[f"{column}_x"].fillna(full_df[f"{column}_y"])
            full_df = full_df.drop([f"{column}_x",f"{column}_y"], axis=1)           

# Sort entries
full_df = reindex(full_df)
# Drop any column that contains only missing values
full_df = full_df.loc[:, (full_df != 0).any(axis=0)]
# Pivot into desired trend_model format
full_df = full_df.reset_index().pivot_table('price', ['date'], 'fsym_id').reset_index().rename(columns={'date':'Dates'})
full_df['Dates'] = full_df['Dates'].dt.date
full_df = full_df.set_index('Dates')
# full_df.to_excel('dataset template.xlsx')
prices = full_df
display(prices)

Prices/2017.csv
Prices/2018.csv
Prices/2019.csv
Prices/2020.csv
Prices/2021.csv


fsym_id,B019MC-R,B01HWF-R,B03993-R,B045ZY-R,B04XY5-R,B054YZ-R,B05Q0H-R,B066VV-R,B06HD3-R,B0GMKD-R,...,XQBP9V-R,XQC8FD-R,XQCN4R-R,XQCWLZ-R,XQDRS9-R,XQDTCV-R,XQF515-R,XQHDVT-R,XQQB5F-R,XR00QK-R
Dates,,,,,,,,,,,,,,,,,,,,,
2017-01-01,14.260,17.084894,0.415000,4.992,21.66,16.18,154.00000,1629.5,16.87,NaN,...,62.313107,96.493210,31.13,52.62,77.75,29400.0,2.184,NaN,NaN,10.1200
2017-01-02,13.860,17.084894,0.415000,5.120,21.66,16.18,154.00000,1629.5,16.87,NaN,...,62.313107,96.493210,31.13,52.62,78.00,29650.0,2.184,NaN,NaN,10.1200
2017-01-03,13.695,17.280151,0.417500,5.230,21.98,16.53,158.22372,1629.5,16.64,NaN,...,62.313107,95.997100,31.63,54.66,77.00,29300.0,2.175,NaN,NaN,10.1592
2017-01-04,14.095,17.377777,0.418333,5.140,21.21,16.91,155.58289,1664.5,16.75,NaN,...,62.313107,96.803276,31.61,55.33,76.75,29300.0,2.186,NaN,NaN,10.1200
2017-01-05,14.160,17.036081,0.419166,5.030,20.21,16.84,156.13126,1643.0,16.71,NaN,...,62.313107,96.493210,31.63,54.45,76.50,29200.0,2.187,NaN,NaN,10.1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-25,12.020,92.800000,0.285000,9.540,23.82,18.18,300.22922,1477.5,28.85,183.1,...,545.360000,104.700000,28.79,16.31,419.00,31050.0,1.371,401.927,1.79,7.7000
2021-11-26,11.980,92.180000,0.285000,8.990,23.79,18.16,285.44190,1473.5,27.22,176.5,...,549.730000,102.420000,25.80,16.41,400.00,31150.0,1.382,385.640,1.77,7.2700
2021-11-27,11.980,92.180000,0.285000,8.990,23.79,18.16,285.44190,1473.5,27.22,176.5,...,549.730000,102.420000,25.80,16.41,400.00,31150.0,1.382,385.640,1.77,7.2700


In [24]:
def run_strat(data, lookback, enter, exit):

###Parameters###
# lookback = 20
# width = 2
# enter = 0.9
# exit = 0.7
# df = prices.copy()
    df = data.copy()

    for name in df.columns:
        df[name + ' SMA'] = np.log(df[name]).rolling(lookback).mean()
    #     df[name + ' UB'] = df[name + ' SMA'] + (np.log(df[name]).rolling(lookback).std() * width) #upper band
    #     df[name + ' LB'] = df[name + ' SMA'] - (np.log(df[name]).rolling(lookback).std() * width) #lower band
       # df[df[name] > df[name + 'UB']].astype(int) * -1 + df[df[name] < df[name + 'LB']].astype(int) 
        df[name + ' Z-Score'] = (np.log(df[name]) - df[name + ' SMA'])/np.log(df[name]).rolling(lookback).std()


        #calculating Short and Long Positions based on the location of Z-Score indicator and the enter-width band. 
        #We unwind our positions when the indicator reaches the exit band

        df[name + ' Sell Position'] = np.where(df[name + ' Z-Score'] > enter, -1, 
                      np.where(df[name + ' Z-Score'] < exit, 0, np.nan))
        df[name + ' Sell Position'] = df[name + ' Sell Position'].ffill().fillna(0)

        df[name + ' Buy Position'] = np.where(df[name + ' Z-Score'] < -1 * enter, 1, 
                          np.where(df[name + ' Z-Score'] > -1 * exit, 0, np.nan))
        df[name + ' Buy Position'] = df[name + ' Buy Position'].ffill().fillna(0)
        df[name + '|Position'] = df[name + ' Buy Position'] + df[name + ' Sell Position']
        df[name + ' Weighted Position'] = df[name + '|Position'] * np.abs(df[name + ' Z-Score'])
        df[name + '|P&L'] = df[name + ' Weighted Position'] * (1 + (df[name].diff().shift(-1)/df[name]))
    df['Total P&L'] = df.filter(regex = '\|P&L').sum(axis = 1)
    df['Cumulative P&L'] = df['Total P&L'].cumsum()
        
    return df

In [34]:
prices.index = pd.to_datetime(prices.index)
prices_group = prices.groupby(prices.index.year)
prices_2020 = prices_group.get_group(2020)

In [35]:
#best parameters to check/only testing on the year 2020
df1 = run_strat(prices_2020,10,2.6,0.6)
df2 = run_strat(prices_2020,15,2.9,0.8)
df3 = run_strat(prices_2020,20,2.8,1.1)
df4 = run_strat(prices_2020,25,2.9,1.3)
df4

fsym_id,B019MC-R,B01HWF-R,B03993-R,B045ZY-R,B04XY5-R,B054YZ-R,B05Q0H-R,B066VV-R,B06HD3-R,B0GMKD-R,...,XQQB5F-R|P&L,XR00QK-R SMA,XR00QK-R Z-Score,XR00QK-R Sell Position,XR00QK-R Buy Position,XR00QK-R|Position,XR00QK-R Weighted Position,XR00QK-R|P&L,Total P&L,Cumulative P&L
Dates,,,,,,,,,,,,,,,,,,,,,
2020-01-01,10.66,6.882772,0.330,5.758,11.560,17.13,288.5000,1498.5,16.010,109.205830,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2020-01-02,10.38,6.873009,0.339,5.818,11.300,17.06,291.9800,1498.5,16.020,111.959330,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2020-01-03,10.96,7.038977,0.337,5.742,11.615,16.85,293.9000,1498.5,16.490,110.877594,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2020-01-04,10.96,7.038977,0.337,5.742,11.615,16.85,293.9000,1498.5,16.490,110.877594,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2020-01-05,10.96,7.038977,0.337,5.742,11.615,16.85,293.9000,1498.5,16.490,110.877594,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,10.28,56.760000,0.230,7.695,17.380,24.23,209.9824,1559.0,14.140,139.592680,...,0.0,1.668622,1.037887,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2020-12-28,10.54,51.330000,0.230,7.885,17.330,25.16,215.9000,1572.5,14.150,139.789350,...,0.0,1.669565,1.076786,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2020-12-29,10.72,51.540000,0.249,8.195,17.070,24.52,214.1000,1587.0,14.025,139.936870,...,0.0,1.669492,0.726420,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [48]:
trend2020 = pd.read_excel('Prices/8kTrendToneBias2020.xlsx')
df_list = [df1,df2,df3,df4]
bbands = pd.DataFrame()
for i,df in enumerate(df_list):
#     positions = df.filter(regex = "\|Position")
#     positions.to_csv('BollingerBands' + str(i) + '.csv')
#     bbands = pd.read_csv('BollingerBands' + str(i) + '.csv').rename(columns={"Dates":'Date'})
#     bbands['Date'] = pd.to_datetime(bbands['Date'])
#     bbands = bbands.melt(id_vars=["Date"], var_name="fsym_id", value_name="BBand")
#     bbands['fsym_id'] = bbands['fsym_id'].replace("|Position","",regex=True)
#     #agreeing long,short positions
#     for i,trend in enumerate([trend2020]):
#         trend_agreeing = trend[trend['trend+tone+bias agree'] == 1]
#         trend_agreeing = trend_agreeing[trend_agreeing['predicted percentage returns'] > 10]
#         trend_agreeing['trend entry date'] = pd.to_datetime(trend_agreeing['trend entry date'])
#         trend_bbands = trend_agreeing.merge(bbands, left_on=['fsym_id', 'trend entry date'], right_on=['fsym_id', 'Date'])
#         trend_bbands['BBands agree'] = trend_bbands.apply(lambda x: (((x['trend short or long'] == 'long') and (x['BBand'] == 1)) or ((x['trend short or long'] == 'short') and (x['BBand'] == -1))), axis=1).astype(int)
#         print(f"{2020} avg. returns for trend+tone trades with 10%+ predicted returns was {round(trend_agreeing['trend actual returns'].mean(),4)}% in {trend_agreeing['trend actual returns'].count()} trades.")
#         print(f"{2020} avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].count()} trades.")
#         print(f"{2020} avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].count()} trades.")
#         print()
        
    #agreeing weight positions
    weight_positions = df.filter(regex = 'Weight')
    weight_positions = weight_positions.fillna(0)
    weight_positions.to_csv('BollingerBandsWeighted' + str(i) + '.csv')
    bbands = pd.read_csv('BollingerBandsWeighted' + str(i) + '.csv').rename(columns={"Dates":'Date'})
    bbands['Date'] = pd.to_datetime(bbands['Date'])
    bbands = bbands.melt(id_vars=["Date"], var_name="fsym_id", value_name="BBand")
    bbands['fsym_id'] = bbands['fsym_id'].replace(" Weighted Position","",regex=True)
    for i,trend in enumerate([trend2020]):
        trend_agreeing = trend[trend['trend+tone+bias agree'] == 1]
        trend_agreeing = trend_agreeing[trend_agreeing['predicted percentage returns'] > 10]
        trend_agreeing['trend entry date'] = pd.to_datetime(trend_agreeing['trend entry date'])
        trend_bbands = trend_agreeing.merge(bbands, left_on=['fsym_id', 'trend entry date'], right_on=['fsym_id', 'Date'])
        trend_bbands['BBands agree'] = trend_bbands.apply(lambda x: (((x['trend short or long'] == 'long') and (x['BBand'] == 1)) or ((x['trend short or long'] == 'short') and (x['BBand'] == -1))), axis=1).astype(int)
        print(f"{2020} avg. returns for trend+tone trades with 10%+ predicted returns was {round(trend_agreeing['trend actual returns'].mean(),4)}% in {trend_agreeing['trend actual returns'].count()} trades.")
        print(f"{2020} avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].count()} trades.")
        print(f"{2020} avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].count()} trades.")
        print()
        print(trend_agreeing)
    
    
    
    
    
    
    
    

2020 avg. returns for trend+tone trades with 10%+ predicted returns was -1.5262% in 3844 trades.
2020 avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was nan% in 0 trades.
2020 avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was -1.5262% in 3844 trades.

         fsym_id  bbg_ticker                                Company  \
3530    V85VP8-R      548 HK  Shenzhen Expressway Co., Ltd. Class H   
5287    LF70RT-R    DNORD DC                         D/S Norden A/S   
8174    P1YJRG-R     9697 JP                       Capcom Co., Ltd.   
12340   BQXYZ7-R     3711 TT       ASE Technology Holding Co., Ltd.   
12946   V5WMVF-R     MCHP US      Microchip Technology Incorporated   
...          ...         ...                                    ...   
744441  MTG5RJ-R     SKAB SS                     Skanska AB Class B   
744859  HY09DC-R     9869 JP                  Kato Sangyo Co., Ltd.   
745468  X434V7-R     WKHS US   

2020 avg. returns for trend+tone trades with 10%+ predicted returns was -1.5262% in 3844 trades.
2020 avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was nan% in 0 trades.
2020 avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was -1.5262% in 3844 trades.

         fsym_id  bbg_ticker                                Company  \
3530    V85VP8-R      548 HK  Shenzhen Expressway Co., Ltd. Class H   
5287    LF70RT-R    DNORD DC                         D/S Norden A/S   
8174    P1YJRG-R     9697 JP                       Capcom Co., Ltd.   
12340   BQXYZ7-R     3711 TT       ASE Technology Holding Co., Ltd.   
12946   V5WMVF-R     MCHP US      Microchip Technology Incorporated   
...          ...         ...                                    ...   
744441  MTG5RJ-R     SKAB SS                     Skanska AB Class B   
744859  HY09DC-R     9869 JP                  Kato Sangyo Co., Ltd.   
745468  X434V7-R     WKHS US   

""
0
1
2
3
4
...
55373
55374
55375
55376


In [45]:
trend2020['bbands agree']

KeyError: 'bbands agree'

In [ ]:
trend2017 = pd.read_excel('Prices/8kTrendToneBias2017.xlsx')
trend2018 = pd.read_excel('Prices/8kTrendToneBias2018.xlsx')
trend2019 = pd.read_excel('Prices/8kTrendToneBias2019.xlsx')
trend2020 = pd.read_excel('Prices/8kTrendToneBias2020.xlsx')
trend2021 = pd.read_excel('Prices/8kTrendToneBias2021.xlsx')
bbands = pd.read_csv('BollingerBands.csv').rename(columns={"Dates":'Date'})
bbands['Date'] = pd.to_datetime(bbands['Date'])
bbands = bbands.melt(id_vars=["Date"], var_name="fsym_id", value_name="BBand")
bbands['fsym_id'] = bbands['fsym_id'].replace("|Position","",regex=True)

for i,trend in enumerate([trend2017,trend2018,trend2019,trend2020,trend2021]):
    trend_agreeing = trend[trend['trend+tone+bias agree'] == 1]
    trend_agreeing = trend_agreeing[trend_agreeing['predicted percentage returns'] > 10]
    trend_agreeing['trend entry date'] = pd.to_datetime(trend_agreeing['trend entry date'])
    trend_bbands = trend_agreeing.merge(bbands, left_on=['fsym_id', 'trend entry date'], right_on=['fsym_id', 'Date'])
    trend_bbands['BBands agree'] = trend_bbands.apply(lambda x: (((x['trend short or long'] == 'long') and (x['BBand'] == 1)) or ((x['trend short or long'] == 'short') and (x['BBand'] == -1))), axis=1).astype(int)
    print(f"{i+2017} avg. returns for trend+tone trades with 10%+ predicted returns was {round(trend_agreeing['trend actual returns'].mean(),4)}% in {trend_agreeing['trend actual returns'].count()} trades.")
    print(f"{i+2017} avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].count()} trades.")
    print(f"{i+2017} avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].count()} trades.")
    print()

In [ ]:
long_pos = positions.apply(lambda x: x[x == 1], axis = 1)
short_pos = positions.apply(lambda x: x[x == -1], axis = 1)

In [ ]:
weight_positions = df.filter(regex = 'Weight')

In [ ]:
weight_positions

In [ ]:
weight_positions.to_csv('BollingerBandsWeighted.csv')

In [ ]:
# trend2017 = pd.read_excel('LSTM-CNN-trend-model/Trend results/Daily8k/8kTrendToneBias2017.xlsx')
# trend2018 = pd.read_excel('LSTM-CNN-trend-model/Trend results/Daily8k/8kTrendToneBias2018.xlsx')
# trend2019 = pd.read_excel('LSTM-CNN-trend-model/Trend results/Daily8k/8kTrendToneBias2019.xlsx')
# trend2020 = pd.read_excel('LSTM-CNN-trend-model/Trend results/Daily8k/8kTrendToneBias2020.xlsx')
# trend2021 = pd.read_excel('LSTM-CNN-trend-model/Trend results/Daily8k/8kTrendToneBias2021.xlsx')
bbands = pd.read_csv('BollingerBandsWeighted.csv').rename(columns={"Dates":'Date'})
bbands['Date'] = pd.to_datetime(bbands['Date'])
bbands = bbands.melt(id_vars=["Date"], var_name="fsym_id", value_name="BBand")
bbands['fsym_id'] = bbands['fsym_id'].replace(" Weighted Position","",regex=True)

for i,trend in enumerate([trend2017,trend2018,trend2019,trend2020,trend2021]):
    trend_agreeing = trend[trend['trend+tone+bias agree'] == 1]
    trend_agreeing = trend_agreeing[trend_agreeing['predicted percentage returns'] > 10]
    trend_agreeing['trend entry date'] = pd.to_datetime(trend_agreeing['trend entry date'])
    trend_bbands = trend_agreeing.merge(bbands, left_on=['fsym_id', 'trend entry date'], right_on=['fsym_id', 'Date'])
    trend_bbands['BBands agree'] = trend_bbands.apply(lambda x: (((x['trend short or long'] == 'long') and (x['BBand'] == 1)) or ((x['trend short or long'] == 'short') and (x['BBand'] == -1))), axis=1).astype(int)
    print(f"{i+2017} avg. returns for trend+tone trades with 10%+ predicted returns was {round(trend_agreeing['trend actual returns'].mean(),4)}% in {trend_agreeing['trend actual returns'].count()} trades.")
    print(f"{i+2017} avg. returns for BBands agreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 1]['trend actual returns'].count()} trades.")
    print(f"{i+2017} avg. returns for BBands disagreeing with trend+tone trades with 10%+ predicted returns was {round(trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].mean(),4)}% in {trend_bbands[trend_bbands['BBands agree'] == 0]['trend actual returns'].count()} trades.")
    print()